![1](ATLASOD.gif)

## Trečia pamoka

Šioje pamokoje mes:
* Iš naujo atrasime Higso bozoną!

In [1]:
import ROOT
%jsroot on
ROOT.gStyle.SetOptStat(0)

Welcome to JupyROOT 6.26/06


In [16]:
# Duomenų failai su dviem fotonais
# Šį kartą panaudosime TChain klasę, kuri leidžia įskaityti daugiau negu vieną failą
# Veikia labai panašiai į TFile/TTree

medis = ROOT.TChain("mini")
nuoroda = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/Data/"
medis.AddFile(nuoroda+"data_A.GamGam.root")
medis.AddFile(nuoroda+"data_B.GamGam.root")
medis.AddFile(nuoroda+"data_C.GamGam.root")
medis.AddFile(nuoroda+"data_D.GamGam.root")

# medis = failas.Get("mini")
#Priminimas: rasti fotonų duomenų šakas galima taip:
medis.Print("pho*")
medis.GetEntries()

7798424

******************************************************************************
*Chain   :mini      : https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/Data/data_A.GamGam.root *
******************************************************************************
******************************************************************************
*Tree    :mini      : mini                                                   *
*Entries :   430344 : Total =       415933847 bytes  File  Size =   81295914 *
*        :          : Tree compression factor =   4.64                       *
******************************************************************************
*Br    0 :photon_n  : photon_n/i                                             *
*Entries :   430344 : Total  Size=    1722197 bytes  File Size  =       9684 *
*Baskets :        2 : Basket Size=    1890304 bytes  Compression= 119.04     *
*............................................................................*
*Br    1 :pho

In [20]:
# Pasitelkiant praeitų dviejų pamokų turiniu, parašykime funkciją kuri nuskaito failą
# ir vietoj dviejų *leptonų* išrenka du *fotonus*
# Bandysime palyginti tikrus su simuliuotais duomenimis.
# Kad veltui nekartoti to paties skaičiavimo, apibrėžkime funkciją
def gauti_masės_histogramą(medis, pavadinimas, n_įvykių, intervalai=30, min_masė=105, max_masė=180):    
    masės_histograma = ROOT.TH1D(pavadinimas, 
                                 "; m [GeV];",
                                 intervalai, min_masė, max_masė)

    # Pastorinkime histogramą kad aiškiau matytųsi grafike
    masės_histograma.SetLineWidth(2)

    pirmas_fotonas = ROOT.TLorentzVector()
    antras_fotonas = ROOT.TLorentzVector()

    n = 0
    # Panašiai, kaip ir pirmoje pamokoje
    for įvykis in medis:
        # Turi būti bent jau du fotonai
        # Jūsų kodas apačioje 
        if įvykis.photon_n < 2:
            continue
            
        gerų_fotonų_indeksų_masyvas = []
        for indeksas in range(įvykis.photon_n):
            if (įvykis.photon_pt[indeksas] > 25000) and (įvykis.photon_isTightID[indeksas]) and \
            (abs(įvykis.photon_eta[indeksas]) < 1.37 or abs(įvykis.photon_eta[indeksas]) > 1.57):
                gerų_fotonų_indeksų_masyvas.append(indeksas)
    
        if len(gerų_fotonų_indeksų_masyvas) != 2:
            continue
            
        pirmo_fotono_indeksas = gerų_fotonų_indeksų_masyvas[0]
        antro_fotono_indeksas = gerų_fotonų_indeksų_masyvas[1]
        #pirmo_fotono_indeksas = 0
        #antro_fotono_indeksas = 1
        
        # Gaukite fotonų judesio kiekius ir energijas
        pirmo_pt = įvykis.photon_pt[pirmo_fotono_indeksas]/1000.
        antro_pt = įvykis.photon_pt[antro_fotono_indeksas]/1000.
        pirmo_E = įvykis.photon_E[pirmo_fotono_indeksas]/1000.
        antro_E = įvykis.photon_E[antro_fotono_indeksas]/1000.
        pirmo_phi = įvykis.photon_phi[pirmo_fotono_indeksas]
        antro_phi = įvykis.photon_phi[antro_fotono_indeksas]
        pirmo_eta = įvykis.photon_eta[pirmo_fotono_indeksas]
        antro_eta = įvykis.photon_eta[antro_fotono_indeksas]

        pirmas_fotonas.SetPtEtaPhiE(pirmo_pt, pirmo_eta, pirmo_phi, pirmo_E)
        antras_fotonas.SetPtEtaPhiE(antro_pt, antro_eta, antro_phi, antro_E)

        # Apskaičiuokite sistemos masę
        sistema = pirmas_fotonas + antras_fotonas
        masė = sistema.M()

        masės_histograma.Fill(masė)
        n += 1
        if n == n_įvykių:
            break

    # Grąžiname histogramą naudotojui
    return masės_histograma

Jokio rezonanso fone akivaizdžiai nesimato... Kad sumažinti, funkcijoje turėtume įgyvendinkite šiuos reikalavimus fotonams: $p_\mathrm{T} > 25\,\mathrm{GeV}, ir neapimti 1.37<|\eta|<1.57$. Taipogi turime pareikalauti, kad fotonai tenkina identifikacijos reikalavimus `isTightID`. Tai turėtų padidinti santykį tarp tikrų bei triukšmo fotonų, to pasekoje ir tarp signalo ir triukšmo įvykių (kvadratu, nes kiekvienam įvykiui reikia dviejų fotonų!)

Užuomina: Iš anksto nežinome, kurie iš fotonų atitinka mūsų reikalavimus. Iki šiol visada naudojome pirmus du fotonus, kurie turėjo indeksus 0 ir 1, tačiau dabar to užtikrinti negalima. Funkcijoje reikėtų įvesti antrą for ciklą, kuris kiekvienam įvykiui patikrintų reikiamas sąlygas. Prieš tai reiktų sukurti švarių indeksų masyvą, kuriame talpinsime informaciją apie švarius fotonus. Python tai galima padaryti taip:  
```
gerų_fotonų_indeksų_masyvas = []
for indeksas in range(įvykis.photon_n):
    if (jūsų tikrinimo kodas):
        gerų_fotonų_indeksų_masyvas.append(indeksas)
```

Toliau funkcijoje galite įvesti gerų fotonų indeksus, kurie atitiks pirmus du indeksus iš gerų fotonų indeksų masyvo (sąrašo):  
```
pirmo_fotono_indeksas = gerų_fotonų_indeksų_masyvas[0]  
antro_fotono_indeksas = gerų_fotonų_indeksų_masyvas[1]
``` 
Tokiu būdu tinkamas fotono savybes (šiuo atveju energiją) galima pasirinkti tokiu būdu: `įvykis.photon_E[pirmo_fotono_indeksas], įvykis.photon_E[antro_fotono_indeksas]`

In [ ]:
# Įgyvendinus fotonų masės gavimo funkciją, nupiešiame fotonų masės spektro grafiką
histograma = gauti_masės_histogramą(medis, "duomenys", -1)

In [ ]:
drobė = ROOT.TCanvas()
drobė.Draw("")
histograma.Draw("PE")
drobė.SaveAs("mgg.png")
histograma.GetEntries()

Turėtumėte gauti kažką panašaus į šį grafiką: